## Connection to Thymio

In [2]:
# Connect to Thymio with ClientAsync
from tdmclient import ClientAsync, aw
client = ClientAsync()
node = await client.wait_for_node()

# Lock Thymio
aw(node.lock())

Node 52ceb7f5-6164-4bf6-b28a-33899f775678

## Importation of libraries

In [3]:
from kalman_filtering.kalman_filter import *
from thymio.control import *
from computer_vision.vision import *
from path_planning.path_plan import *
from navigation.collision_avoidance import *

## Main script

In [ ]:
map = Map()
if P_VISION:
    print(P_SETUP)
    print("Place the Thymio and the destination, check the obstacles and the corners. Then, press S.")

while True:
    map.update(True)
    cv.waitKey(0)
    if cv.waitKey(20) & 0xFF==ord('s'):
        if map.get_found_corners():
            map.update()
            break
        elif P_VISION:
            print("USER: Must find corners before start")

if P_VISION: print(P_START)
if P_VISION: map.info()
await node.wait_for_variables()

u_old = np.zeros(2).T
mu_predict_old = np.array([map.robot[0], map.robot[1], u_old[0], u_old[1]]).T
cov_predict_old = 100*np.eye(4)
view_blocked = False
active = True
end = False
start = True
path = None
segment_index = 0
obstacle = 0
error_est= [10,10]

#print("mu_predict_old:", mu_predict_old)

while active == True:
    active = await stop_thymio(node)

    map.update()
    
    # State variables measurement
    await node.wait_for_variables()
    vel_meas = velocity_with_odometry(node, map.robot[2])

    # Filtering
    y = np.array([map.robot[0], map.robot[1], vel_meas[0], vel_meas[1]]).T
    x_est, cov_est = kalman_filter(y, u_old, mu_predict_old, cov_predict_old, view_blocked)
    map.pose_est = np.array([x_est[0], x_est[1], map.robot[2]])
    error_est = np.array([cov_est[0,0], cov_est[1,1], 0])
    print("Pose estimation of the Thymio:", map.pose_est)
   
    # Global Navigation
    if start == True:
        ex_path, corners = compute_visibility_matrix(map.pose_est[0:2], map.destination[0:2], map.obstacles)
        map.obstacles_lines = possible_lignes(ex_path, corners)
        map.target_lines = a_star_search(corners, ex_path)
        path = path_functions(map.target_lines)
        start = False

    # Local Navigation
    u, obstacle, segment = await collision_avoidance(path, segment_index, map.robot, node) # Zacharie


    if not obstacle:
        u, segment_index, end = path_direction(map.pose_est, path, segment_index)

    await node.set_variables(motors(int(u[0]), int(u[1])))

    u_old = u.T
    cov_predict_old = cov_est
    mu_predict_old = x_est.T
    # obstacle_old = obstacle

    if cv.waitKey(20) & 0xFF==ord('p'):
        map.vision_stop()
        break

=====[   SETUP      ]===============
Place the Thymio and the destination, check the obstacles and the corners. Then, press S.
ERROR: Could not detect the four aruco corners: only detected [[2]
 [1]
 [0]
 [5]
 [4]]
=====[   START      ]===============
=====[   MAP INFO   ]===============
* FOUND CORNERS: True

* FOUND ROBOT: True
Position: [47, 263]
Orientation: 0.17065477450758865

* FOUND DESTINATION: True
Position: [561, 287]

* OBSTACLES: 3

mu_predict_old: [ 47. 263.   0.   0.]
Path_reached


error: OpenCV(4.10.0) :-1: error: (-5:Bad argument) in function 'line'
> Overload resolution failed:
>  - Can't parse 'pt1'. Sequence item with index 0 has a wrong type
>  - Can't parse 'pt1'. Sequence item with index 0 has a wrong type


## Motors control test

In [ ]:
active = True
vel_meas = []

while active:
    await node.wait_for_variables()
    #vel_meas.append(velocity_with_odometry(node, 0)[0])
    await node.set_variables(motors(-100, 100))
    active = await stop_thymio(node)

await node.set_variables(motors(0, 0)) 

vel_meas_mean = np.mean(vel_meas[-100:])
vel_meas_var = np.var(vel_meas[-100:])
print("Velocity measured:", vel_meas)
print("Velocity mean:", vel_meas_mean)
print("Velocity variance:", vel_meas_var)

Center button pressed. Stopping the algorithm.
Velocity measured: []
Velocity mean: nan
Velocity variance: nan


c:\Users\samyg\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\samyg\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Users\samyg\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\fromnumeric.py:3787: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\samyg\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
c:\Users\samyg\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.

In [ ]:
node.var

{'_fwversion': [14, 0],
 '_id': [-27410],
 '_imot': [-17, -189],
 '_integrator': [689, -1116],
 '_productId': [8],
 '_vbat': [718, 717],
 'acc': [1, -1, 24],
 'acc._tap': [32],
 'button.backward': [0],
 'button.center': [1],
 'button.forward': [0],
 'button.left': [0],
 'button.right': [0],
 'buttons._mean': [10655, 9142, 13730, 13675, 15078],
 'buttons._noise': [57, 46, 60, 53, 80],
 'buttons._raw': [10641, 9140, 13538, 13663, 15020],
 'event.args': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'event.source': [-27410],
 'leds.bottom.left': [0, 0, 0],
 'leds.bottom.right': [0, 0, 0],
 'leds.circle': [0, 0, 0, 0, 0, 0, 0, 0],
 'leds.top': [0, 0, 0],
 'mic._mean': [261],
 'mic.intensity': [29],
 'mic.threshold': [0],
 'motor.left.pwm': [329],
 'motor.left.speed': [-72],
 'motor.left.target': [-100],
 'motor.right.pwm': [-419],
 'motor.right.speed': [100],
 'motor.right.targ